In [1]:
using Gen

# Involution MCMC in Gen

We'll use Gen's involution-based MCMC to sample from a simple, discrete distribution: the geometric distribution.

In [3]:
@gen function simple_model()
    x = @trace(geometric(0.05), :x)
    return x
end

DynamicDSLFunction{Any}(Dict{Symbol,Any}(), Dict{Symbol,Any}(), Type[], ##simple_model#371, Bool[], false)

## Inference: Metropolis-Hastings

Define a proposal distribution. For illustrative purposes, we implement a proposal that makes random choices **not appearing in the model itself.**

This situation occurs frequently when we have distributions over complicated objects. For example, if we were dealing with distributions over directed graphs, then our proposal distribution might choose random edge deletions/additions/reversals -- whereas the model itself never makes such choices.

It's silly to use involutions in our geometric distribution example, but we'll use the insights we gain here in order to tackle a more challenging task (i.e., directed graphs).

In [6]:
@gen function random_walk_proposal(cur_tr)
    # do we take a step at all?
    a = @trace(uniform_discrete(0, 1), :a)
    # do we go left or right?
    x = @trace(uniform_discrete(-1*a, a), :x)
    return x
end

DynamicDSLFunction{Any}(Dict{Symbol,Any}(), Dict{Symbol,Any}(), Type[Any], ##random_walk_proposal#373, Bool[0], false)

In this proposal distribution, `a` is a random choice. But there is no corresponding choice made in our model, `simple_model`.

What happens if we naively plug this proposal into Gen's Metropolis-Hastings function?

In [5]:
# generate a trace
tr, _ = Gen.generate(simple_model, ())
println(get_choices(tr))

│
└── :x : 8



In [7]:
# call metropolis hastings with our trace and the random walk proposal
Gen.metropolis_hastings(tr, random_walk_proposal, ())

ErrorException: Did not visit all constraints

A straightforward application of Gen's three-argument `metropolis_hastings` fails!

`Did not visit all constraints`

What's going on here? What did we do wrong?

## The Problem

The stack trace tells us that the problem occurs when `metropolis_hastings` makes a call to `update`. 

More precisely, the problem is that `metropolis_hastings` tries to update the model's trace with *all of the random choices* made by the proposal.

So the issue is this simple-minded invocation of `update`.

## The Solution

Gen extends `metropolis_hastings` to allow a fourth `involution` argument. This extended version addresses our problem.

Concretely, it should allow us to use the `random_walk_proposal` we've defined. As long as we're able to define the involution correctly, that is.

## The Involution Function

The following mapping describes the core logic of the involution:

$$ (\text{current trace, proposed random choices})  \ \rightarrow  \ (\text{proposed trace, backward random choices})$$

That is, the involution must
1. receive the current trace, and a set of random choices for updating that trace;
2. return the updated trace that results from those updates; and the set of random choices which *would reset the proposed trace **back** to the current trace*.

This function is called an involution because, mathematically, an involution is a *bijection which, applied twice, is the identity.*

The above mapping is in fact an involution -- if we applied it twice, we would end up with the (current trace, proposed random choices) just as before.

This involution appears in Metropolis-Hastings as part of the calculation of acceptance probabilities:

$$\alpha = \text{min} \left(1, \frac{q(x|x^\prime)}{q(x^\prime | x)} \cdot \frac{P(x^\prime)}{P(x)} \right) $$

More precisely: it contains sufficient information to compute the ratio $q(x | x^\prime) / q(x^\prime | x)$.

## Implementing an involution

In [ ]:
new_choices_mod = Gen.choicemap()
new_choices_mod[:z] = new_choices[:z]

In [ ]:
new_tr, _, _, _ = Gen.update(tr, (0.0, 1.0), (), new_choices_mod)
#Gen.assess(dpmg_proposal, () )

In [ ]:
function dumb_involution(trace, fwd_choices, fwd_retval, proposal_args)
    
    subchoice = Gen.choicemap()
    subchoice[:z] = fwd_choices[:z]
    
    new_tr, weight, _, discard = Gen.update(trace, get_args(trace), (), subchoice)
    
    fwd_a = fwd_choices[:a]
    bwd_choices
    
    return new_tr, bwd_choices, weight
end
#cm = Gen.choicemap()
#cm[:z] = new_tr[:z]
#cm[:a] = new_tr[:a]
#w, r = Gen.assess(dpmg_proposal, (tr,), cm)

In [ ]:
Gen.assess(dpmg_proposal, (new_tr,), new_choices_mod)

In [ ]:
println(get_choices(new_tr))

In [ ]:
print(log.([0.25; 0.5; 0.75]))

# Github issue writeup:

Suppose you have a distribution over complicated objects -- e.g., directed graphs.
* You know the distribution's (unnormalized) density. E.g., for directed graphs we might have
    - ![image](https://user-images.githubusercontent.com/14325161/67121705-c0f07400-f1b1-11e9-9a1c-29815e71deaf.png)
* You **don't** have a generative description of the distribution. E.g., I can't think of a probabilistic program that explicitly generates samples from ![image](https://user-images.githubusercontent.com/14325161/67121746-db2a5200-f1b1-11e9-82f8-ee87fff2af86.png)
* You want to use this "difficult" distribution as a prior in your `Gen` model.

One way to accomplish this:
1. Define a `Distribution` type corresponding to  ![image](https://user-images.githubusercontent.com/14325161/67121746-db2a5200-f1b1-11e9-82f8-ee87fff2af86.png). Its `random` function would simply yield a reasonable initial value, in lieu of an actual random sample.
2. Use MCMC with a suitable proposal distribution (e.g., one which steps through the space of directed graphs via edge addition/deletion/reversal).

That is: the initial call to `generate` would populate the address `:G` with a reasonable initial graph. And the subsequent Metropolis Hastings updates -- informed by a suitable proposal -- would yield samples from the target distribution (conditioned on whatever observations have been made of other variables).

When I do this, I run into the following issue:
* My proposal distribution is a `@gen function` which makes several random choices -- one of which is the updated `:G`. The other random choices are specific to the proposal distribution, and play no role in the actual model. For example, my proposal
    - chooses a vertex at random: `:v`
    - decides whether or not to add an edge: `:add_edge`
    - if we choose to add an edge, then choose a non-neighbor at random: `:u`
    - etc.
* These random choices  